In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Text Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\Text Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: str
    unzip_dir: Path

In [6]:
from src.Text_Summarization.constants import *
from src.Text_Summarization.utils.common import read_yaml, create_directories

In [7]:
import sys
sys.path

['e:\\Projects\\Text Summarization\\venv\\python310.zip',
 'e:\\Projects\\Text Summarization\\venv\\DLLs',
 'e:\\Projects\\Text Summarization\\venv\\lib',
 'e:\\Projects\\Text Summarization\\venv',
 '',
 'e:\\Projects\\Text Summarization\\venv\\lib\\site-packages',
 'e:\\projects\\text summarization',
 'e:\\Projects\\Text Summarization\\venv\\lib\\site-packages\\win32',
 'e:\\Projects\\Text Summarization\\venv\\lib\\site-packages\\win32\\lib',
 'e:\\Projects\\Text Summarization\\venv\\lib\\site-packages\\Pythonwin']

In [8]:
BASE_DIR

WindowsPath('E:/Projects/Text Summarization')

In [9]:
CONFIG_FILE_PATH

WindowsPath('E:/Projects/Text Summarization/config/config.yaml')

In [10]:
class ConfigrationManager:
    def __init__(self, 
                 params_filepath= PARAMS_FILE_PATH,
                 config_filepath= CONFIG_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        #print(config.root_dir)
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from src.Text_Summarization.logging import logger
from src.Text_Summarization.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
            url = self.config.source_url,
            filename = self.config.local_data_file
            )
            print(filename,headers)
            logger.info(f"{filename} downloaded with headers{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unizip_path = self.config.unzip_dir
        os.makedirs(unizip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unizip_path)

In [15]:
from src.Text_Summarization.exception import CustomException
import sys

try:
    config = ConfigrationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-08-12 14:23:29,187 ] 23 TextSummarizationLogger - INFO - yaml file: E:\Projects\Text Summarization\config\config.yaml loaded successfully
[ 2024-08-12 14:23:29,197 ] 23 TextSummarizationLogger - INFO - yaml file: params.yaml loaded successfully
[ 2024-08-12 14:23:29,197 ] 41 TextSummarizationLogger - INFO - created directory at: artifacts
[ 2024-08-12 14:23:29,197 ] 41 TextSummarizationLogger - INFO - created directory at: artifacts/data_ingestion
artifacts/data_ingestion/data.zip Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 212A:08EE:1F9927:2ADD9E:66B9CD88
Accept-Ranges: bytes
Date: Mon, 12 Aug 2024 08:53:31 GMT
Via: 1.1 varni

CustomException: Error occured in Python Script name [C:\Users\HP\AppData\Local\Temp\ipykernel_4288\3943882422.py] line number [9] error message [type object 'zip' has no attribute 'ref']